In [1]:
import pandas as pd
import numpy as np
import sklearn
import torch.nn as nn
import sklearn.model_selection
from sklearn import linear_model
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
import pickle
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
pd.reset_option('display.max_rows')

In [2]:
def add_target(team):
    team['target'] = team['won'].shift(-1)
    return team

def rest_days(team):
    team['date_time'] = pd.to_datetime(team['date'])
    team['rest'] = (team['date_time'] - team['date_time'].shift(1)).dt.days
    team['rest'] = team['rest'].fillna(0)
    team['rest'] = team['rest'].astype(int)
    del team['date_time']
    return team

def winrate(team):
    total = team['Wins'] + team['Losses']
    total_opp = team['Wins_opp'] + team['Losses_opp']
    team['winrate'] = team['Wins'] / total
    team['winrate_opp'] = team['Wins_opp'] / total_opp
    return team

def differential(team):
    team['differential'] = team['Total'] - team['Total_opp']
    return team

def find_team_exp_average_5(team):
    numeric_columns = team.select_dtypes(include=np.number)
    rolling = numeric_columns.ewm(span=5, adjust=False).mean()
    return rolling

def find_team_exp_average_9(team):
    numeric_columns = team.select_dtypes(include=np.number)
    rolling = numeric_columns.ewm(span=9, adjust=False).mean()
    return rolling

def find_team_exp_average_12(team):
    numeric_columns = team.select_dtypes(include=np.number)
    rolling = numeric_columns.ewm(span=12, adjust=False).mean()
    return rolling

def find_team_average_15(team):
    numeric_columns = team.select_dtypes(include=np.number)
    rolling = numeric_columns.rolling(15).mean()
    return rolling

def find_team_average_10(team):
    numeric_columns = team.select_dtypes(include=np.number)
    rolling = numeric_columns.rolling(10).mean()
    return rolling

def find_team_average_5(team):
    numeric_columns = team.select_dtypes(include=np.number)
    rolling = numeric_columns.rolling(5).mean()
    return rolling

def find_team_average_3(team):
    numeric_columns = team.select_dtypes(include=np.number)
    rolling = numeric_columns.rolling(3).mean()
    return rolling

def rolling(data):
    df_rolling_3 = data[list(valid_columns) + ['Teams','won', "season"]]
    df_rolling_3 = df_rolling_3.groupby(['Teams', 'season'], group_keys = False).apply(find_team_average_3)
    df_rolling_5 = data[list(valid_columns) + ['Teams','won', "season"]]
    df_rolling_5 = df_rolling_5.groupby(['Teams', 'season'], group_keys = False).apply(find_team_average_5)
    df_rolling_10 = data[list(valid_columns) + ['Teams','won', "season"]]
    df_rolling_10 = df_rolling_10.groupby(['Teams', 'season'], group_keys = False).apply(find_team_average_10)
    df_rolling_15 = data[list(valid_columns) + ['Teams','won', "season"]]
    df_rolling_15 = df_rolling_15.groupby(['Teams', 'season'], group_keys = False).apply(find_team_average_15)
    df_exp_rolling_5 = data[list(valid_columns) + ['Teams','won', "season"]]
    df_exp_rolling_5 = df_exp_rolling_5.groupby(['Teams', 'season'], group_keys = False).apply(find_team_exp_average_5)
    df_exp_rolling_9 = data[list(valid_columns) + ['Teams','won', "season"]]
    df_exp_rolling_9 = df_exp_rolling_9.groupby(['Teams', 'season'], group_keys = False).apply(find_team_exp_average_9)
    df_exp_rolling_12 = data[list(valid_columns) + ['Teams','won', "season"]]
    df_exp_rolling_12 = df_exp_rolling_12.groupby(['Teams', 'season'], group_keys = False).apply(find_team_exp_average_12)
    exp_rolling_columns_5 = [f"{col}_exp_5" for col in df_exp_rolling_5.columns]
    exp_rolling_columns_9 = [f"{col}_exp_9" for col in df_exp_rolling_9.columns]
    exp_rolling_columns_12 = [f"{col}_exp_12" for col in df_exp_rolling_12.columns]
    rolling_columns_15 = [f"{col}_15" for col in df_rolling_15.columns]
    rolling_columns_10 = [f"{col}_10" for col in df_rolling_10.columns]
    rolling_columns_5 = [f"{col}_5" for col in df_rolling_5.columns]
    rolling_columns_3 = [f"{col}_3" for col in df_rolling_3.columns]
    df_exp_rolling_12.columns = exp_rolling_columns_12
    df_exp_rolling_9.columns = exp_rolling_columns_9
    df_exp_rolling_5.columns = exp_rolling_columns_5
    df_rolling_15.columns = rolling_columns_15
    df_rolling_10.columns = rolling_columns_10
    df_rolling_5.columns = rolling_columns_5
    df_rolling_3.columns = rolling_columns_3
    df = pd.concat([data, df_rolling_3, df_rolling_5, df_rolling_10, df_rolling_15, df_exp_rolling_5,df_exp_rolling_9, df_exp_rolling_12], axis=1)
    # df_exp_rolling_5,df_exp_rolling_9, df_exp_rolling_12
    return df

def ratio(feature):
    feature_opp = 'OPP_' + str(feature)
    free = nba[feature] / nba[feature_opp]
    return free

def ratios(nba):
    regard = []
    disregard = [col for col in nba.columns if "OPP_" in col]
    for col in disregard:
        col = col[4:100]
        if col in nba.columns:
            regard.append(col)
    nba_ratio = nba[regard].apply(ratio)
    nba_ratios_columns = [f"{col}_ratio" for col in nba_ratio.columns]
    nba_ratio.columns
    return regard

def shift_col(team, col_name):
    next_col = team[col_name].shift(-1)
    return next_col

def add_col(df, col_name):
    return df.groupby("Teams", group_keys=False).apply(lambda x: shift_col(x, col_name))

def date_change(datetime_str):
    # Parse the datetime string into a datetime object
    datetime_obj = datetime.strptime(datetime_str, '%m/%d/%Y')

    # Format the datetime object into a new string structure
    new_datetime_str = datetime_obj.strftime('%Y-%m-%d')

    return new_datetime_str

def haircut(df, date):
    df[date] = df[date].str[:10]
    return df

def convert_date_format(df):
    # Create a boolean mask to identify values in the "m/d/y" format
    mask = df['Date'].str.contains(r'\d{1,2}/\d{1,2}/\d{2}')
    
    # Apply the conversion only to values that match the mask
    df.loc[mask, 'Date'] = nba.loc[mask, 'Date'].apply(date_change)
    return df

def spread(df):
    # construct spread between home and away rankings
    ranks = [i for i in df.columns if 'RANK' in i]
    ranks_home = [i for i in ranks if '_x' in i]
    ranks_away = [i for i in ranks if '_y' in i]
    spread_columns_names = [col[:-1] for col in ranks_home]
    
    spread_columns = []
    for base_name in spread_columns_names:
        spread_column = df[f'{base_name}x'] - df[f'{base_name}y']
        spread_column.name = f'{base_name}spread'
        spread_columns.append(spread_column)
        
    spread_df = pd.concat(spread_columns, axis=1)
    return spread_df

def last_season(df):
    # construct winrate from last season
    df['last_season_winrate'] = df.groupby('Teams', 'season')['winrate'].last()

In [3]:
folder_path = "/Users/benjamincheng/Documents/GitHub/Sports-Betting/data/raw_data/NBA_2018_2024.csv"
#folder_path = "/Users/liqingyang/Documents/GitHub/sports_trading/sports_betting/data/raw_data/NBA_2018_2024.csv"

df = pd.read_csv(folder_path, index_col=0)

folder_path = "/Users/benjamincheng/Documents/GitHub/Sports-Betting/nba_api/data/teams_stats/processed_cum_2018_2024.csv"
#folder_path = "/Users/liqingyang/Documents/GitHub/sports_trading/sports_betting/nba_api/data/teams_stats/processed_cumulative_season_stats_2019_2024.csv"
nba = pd.read_csv(folder_path, index_col=0)

folder_path = "/Users/benjamincheng/Documents/GitHub/Sports-Betting/data/odds_data/2021_01_20_onward.csv"
#folder_path = "/Users/liqingyang/Documents/GitHub/sports_trading/sports_betting/data/odds_data/2021_01_20_onward.csv"
odds = pd.read_csv(folder_path, index_col=0)

In [4]:
df

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,orb,drb,trb,ast,stl,blk,tov,pf,pts,ts%,efg%,3par,ftr,orb%,drb%,trb%,ast%,stl%,blk%,tov%,usg%,ortg,drtg,fgmaxes,fgamaxes,fg%maxes,3pmaxes,3pamaxes,3p%maxes,ftmaxes,ftamaxes,ft%maxes,orbmaxes,drbmaxes,trbmaxes,astmaxes,stlmaxes,blkmaxes,tovmaxes,pfmaxes,ptsmaxes,+/-maxes,ts%maxes,efg%maxes,3parmaxes,ftrmaxes,orb%maxes,drb%maxes,trb%maxes,ast%maxes,stl%maxes,blk%maxes,tov%maxes,usg%maxes,ortgmaxes,drtgmaxes,Teams,Total,Pace,eFG%,TOV%,ORB%,FT/FGA,ORtg,Wins,Losses,home,mp_opp,fg_opp,fga_opp,fg%_opp,3p_opp,3pa_opp,3p%_opp,ft_opp,fta_opp,ft%_opp,orb_opp,drb_opp,trb_opp,ast_opp,stl_opp,blk_opp,tov_opp,pf_opp,pts_opp,ts%_opp,efg%_opp,3par_opp,ftr_opp,orb%_opp,drb%_opp,trb%_opp,ast%_opp,stl%_opp,blk%_opp,tov%_opp,usg%_opp,ortg_opp,drtg_opp,fgmaxes_opp,fgamaxes_opp,fg%maxes_opp,3pmaxes_opp,3pamaxes_opp,3p%maxes_opp,ftmaxes_opp,ftamaxes_opp,ft%maxes_opp,orbmaxes_opp,drbmaxes_opp,trbmaxes_opp,astmaxes_opp,stlmaxes_opp,blkmaxes_opp,tovmaxes_opp,pfmaxes_opp,ptsmaxes_opp,+/-maxes_opp,ts%maxes_opp,efg%maxes_opp,3parmaxes_opp,ftrmaxes_opp,orb%maxes_opp,drb%maxes_opp,trb%maxes_opp,ast%maxes_opp,stl%maxes_opp,blk%maxes_opp,tov%maxes_opp,usg%maxes_opp,ortgmaxes_opp,drtgmaxes_opp,Teams_opp,Total_opp,Pace_opp,eFG%_opp,TOV%_opp,ORB%_opp,FT/FGA_opp,ORtg_opp,Wins_opp,Losses_opp,home_opp,season,date,won
0,240.0,36.0,88.0,0.409,8.0,32.0,0.250,19.0,25.0,0.760,9.0,37.0,46.0,24.0,11.0,4.0,10.0,24.0,99.0,0.500,0.455,0.364,0.284,18.0,80.4,47.9,66.7,11.1,6.6,9.2,100.0,99.7,102.7,11.0,23.0,1.000,4.0,9.0,0.500,5.0,7.0,1.000,4.0,9.0,10.0,10.0,4.0,2.0,3.0,5.0,25.0,8.0,0.798,1.000,1.000,2.000,10.5,26.8,13.7,46.5,9.9,4.5,34.7,29.9,129.0,112.0,BOS,99,99.3,0.455,9.2,18.0,0.216,99.7,0,1,0,240.0,38.0,83.0,0.458,5.0,22.0,0.227,21.0,25.0,0.840,9.0,41.0,50.0,19.0,3.0,4.0,17.0,25.0,102.0,0.543,0.488,0.265,0.301,19.6,82.0,52.1,50.0,3.0,7.1,15.3,100.0,102.7,99.7,12.0,19.0,0.667,1.0,5.0,0.333,6.0,7.0,1.000,3.0,15.0,16.0,9.0,2.0,2.0,4.0,4.0,29.0,9.0,0.698,0.667,0.500,1.000,11.0,35.0,19.4,43.7,3.8,6.0,31.6,27.3,138.0,107.0,CLE,102,99.3,0.488,15.3,19.6,0.253,102.7,1,0,1,2018,2017-10-17,False
1,240.0,38.0,83.0,0.458,5.0,22.0,0.227,21.0,25.0,0.840,9.0,41.0,50.0,19.0,3.0,4.0,17.0,25.0,102.0,0.543,0.488,0.265,0.301,19.6,82.0,52.1,50.0,3.0,7.1,15.3,100.0,102.7,99.7,12.0,19.0,0.667,1.0,5.0,0.333,6.0,7.0,1.000,3.0,15.0,16.0,9.0,2.0,2.0,4.0,4.0,29.0,9.0,0.698,0.667,0.500,1.000,11.0,35.0,19.4,43.7,3.8,6.0,31.6,27.3,138.0,107.0,CLE,102,99.3,0.488,15.3,19.6,0.253,102.7,1,0,1,240.0,36.0,88.0,0.409,8.0,32.0,0.250,19.0,25.0,0.760,9.0,37.0,46.0,24.0,11.0,4.0,10.0,24.0,99.0,0.500,0.455,0.364,0.284,18.0,80.4,47.9,66.7,11.1,6.6,9.2,100.0,99.7,102.7,11.0,23.0,1.000,4.0,9.0,0.500,5.0,7.0,1.000,4.0,9.0,10.0,10.0,4.0,2.0,3.0,5.0,25.0,8.0,0.798,1.000,1.000,2.000,10.5,26.8,13.7,46.5,9.9,4.5,34.7,29.9,129.0,112.0,BOS,99,99.3,0.455,9.2,18.0,0.216,99.7,0,1,0,2018,2017-10-17,True
2,240.0,47.0,97.0,0.485,15.0,41.0,0.366,13.0,19.0,0.684,10.0,33.0,43.0,28.0,9.0,5.0,12.0,16.0,122.0,0.579,0.562,0.423,0.196,22.2,84.6,51.2,59.6,8.8,10.0,10.2,100.0,119.6,118.6,10.0,23.0,0.667,4.0,9.0,0.667,6.0,8.0,0.750,3.0,7.0,8.0,11.0,2.0,2.0,3.0,4.0,27.0,20.0,0.859,0.889,0.667,0.667,9.6,26.1,13.8,42.9,2.8,6.5,18.2,31.2,152.0,126.0,HOU,122,102.0,0.562,10.2,22.2,0.134,119.6,1,0,0,240.0,43.0,80.0,0.538,16.0,30.0,0.533,19.0,21.0,0.905,6.0,35.0,41.0,34.0,5.0,9.0,17.0,25.0,121.0,0.678,0.638,0.375,0.263,15.4,77.8,48.8,79.1,4.9,16.1,16.0,100.0,118.6,119.6,8.0,18.0,1.000,6.0,9.0,0.857,4.0,5.0,1.000,1.0,10.0,11.0,13.0,2.0,4.0,8.0,4.0,23.0,11.0,1.218,1.222,0.778,1.000,13.1,38.6,22.7,57.1,6.1,9.1,31.7,32.6,250.0,127.0,GSW,121,102.0,0.638,16.0,15.4,0.238,118.6,0,1,1,2018,2017-10-17,True
3,240.0,43.0,80.0,0.538,16.0,30.0,0.533,19.0,21.0,0.905,6.0,35.0,41.0,34.0,5.0,9.0,17.0,25.0,121.0,0.678,0.638,0.375,0.263,15.4,77.8,48.8,79.1,4.9,16.1,16.0,100.0,118.6,119.6,8.0,18.0,1.000,6.0,9.0,0.857,4.0,5.0,1.000,1.0,10.0,11.0,13.0,2.0,4.0,8.0,4.0,23.0,11.0,1.218,1.222,0.778,1.000,13.1,38.6,22.7,57.1,6.1,9.1,31.7,32.6

In [5]:
nba[nba['Teams'] == 'PHO'].iloc[-1]['Date']

'2024-04-06'

In [6]:
# nba dataframe does not include the 2018 season
df = df[~df['season'].isin([2018])]
df = df.reset_index(drop=True)
df = haircut(df, 'date')

# rename nba columns to match df
nba = haircut(nba, 'Date')
nba = convert_date_format(nba)
nba.rename(columns={'Date': 'date'}, inplace=True)

#rename odds columns to match df
odds.rename(columns={'Timestamp': 'date_next'}, inplace=True)

# construct winrate for team
df = winrate(df)
# construct differential points
df = differential(df)
# construct target
df = df.groupby("Teams", group_keys=False).apply(add_target)
# construct resting
df = df.groupby(["Teams",'season'], group_keys=False).apply(rest_days)
# games yet to play are 2
df.loc[pd.isnull(df['target']), 'target'] = 2
# convert win/loss to 1/0
df['target'] = df['target'].astype(int)

# remove metadata and target for df
removed = ['target', 'date', 'Teams_opp', 'Teams',
           'season','won', 'Wins', 'Losses', 
           'Wins_opp', 'Losses_opp']
valid_columns = df.columns[~df.columns.isin(removed)]

# scale the data for df
scaler = MinMaxScaler()
df[valid_columns] = scaler.fit_transform(df[valid_columns])

# construct rolling features to df
df = rolling(df).copy()
df = df.dropna()

# remove metadata for nba ranking 
removed = ['date', 'Teams']
valid_columns = nba.columns[~nba.columns.isin(removed)]

# scale the ranking data
scaler = MinMaxScaler()
nba[valid_columns] = scaler.fit_transform(nba[valid_columns])

# remove metadata for odds data
# removed = list(odds.columns[odds.dtypes == 'object'])
# valid_columns = odds.columns[~nba.columns.isin(removed)]

# scale the odds data
# scaler = MinMaxScaler()
# odds[valid_columns] = scaler.fit_transform(odds[valid_columns])

# construct current game metadata for df
df['home_next'] = add_col(df, 'home')
df['team_next_opp'] = add_col(df, 'Teams_opp')
df['date_next'] = add_col(df, 'date')
df = df.copy()

full = pd.merge(df, nba, on=['Teams', 'date'], how='left') # problem here 

/var/folders/99/zv3w4flx1598dxlknc099kfr0000gn/T/ipykernel_48072/1808437805.py:19: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby("Teams", group_keys=False).apply(add_target)
/var/folders/99/zv3w4flx1598dxlknc099kfr0000gn/T/ipykernel_48072/1808437805.py:21: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(["Teams",'season'], group_keys=False).apply(rest_days)
/var/folders/99/zv3w

In [7]:
full[full['Teams'] == 'PHO'].iloc[-1]['date']

'2024-04-05'

In [8]:
full

mp        fg       fga       fg%        3p       3pa       3p%  \
0      0.0  0.409091  0.333333  0.480861  0.481481  0.433333  0.450194   
1      0.0  0.409091  0.466667  0.382775  0.370370  0.283333  0.485123   
2      0.0  0.454545  0.633333  0.327751  0.222222  0.300000  0.280724   
3      0.0  0.272727  0.350000  0.296651  0.185185  0.216667  0.304010   
4      0.0  0.545455  0.483333  0.526316  0.074074  0.166667  0.169470   
...    ...       ...       ...       ...       ...       ...       ...   
11172  0.0  0.340909  0.400000  0.346890  0.259259  0.266667  0.358344   
11173  0.0  0.477273  0.483333  0.449761  0.555556  0.483333  0.474774   
11174  0.0  0.386364  0.166667  0.602871  0.333333  0.283333  0.437257   
11175  0.0  0.409091  0.450000  0.392344  0.333333  0.416667  0.316947   
11176  0.0  0.545455  0.283333  0.703349  0.518519  0.450000  0.469599   

             ft       fta       ft%       orb       drb       trb       ast  \
0      0.690476  0.705882  0.721845  0.344828  0.363636  0.384615  0.380952   
1      0.428571  0.352941  0.934871  0.448276  0.431818  0.500000  0.476190   
2      0.357143  0.411765  0.603799  0.551724  0.318182  0.461538  0.214286   
3      0.166667  0.117647  1.000000  0.241379  0.545455  0.480769  0.190476   
4      0.190476  0.274510  0.440977  0.413793  0.477273  0.519231  0.357143   
...         ...       ...       ...       ...       ...       ...       ...   
11172  0.309524  0.352941  0.611940  0.517241  0.295455  0.423077  0.309524   
11173  0.000000  0.019608  0.321574  0.172414  0.159091  0.115385  0.404762   
11174  0.571429  0.588235  0.712347  0.379310  0.477273  0.500000  0.285714   
11175  0.142857  0.176471  0.548168  0.379310  0.295455  0.346154  0.380952   
11176  0.190476  0.254902  0.491180  0.206897  0.409091  0.346154  0.523810   

            stl   blk       tov       pf       pts       ts%      efg%  \
0      0.227273  0.00  0.275862  0.56250  0.518519  0.581858  0.524291   
1      0.363636  0.30  0.413793  0.43750  0.388889  0.426991  0.392713   
2      0.454545  0.10  0.482759  0.71875  0.361111  0.274336  0.293522   
3      0.363636  0.15  0.724138  0.31250  0.129630  0.243363  0.271255   
4      0.409091  0.15  0.655172  0.40625  0.333333  0.369469  0.425101   
...         ...   ...       ...      ...       ...       ...       ...   
11172  0.409091  0.10  0.586207  0.37500  0.259259  0.316372  0.334008   
11173  0.409091  0.10  0.206897  0.56250  0.324074  0.426991  0.502024   
11174  0.136364  0.35  0.586207  0.15625  0.416667  0.641593  0.597166   
11175  0.500000  0.15  0.586207  0.15625  0.268519  0.338496  0.390688   
11176  0.500000  0.55  0.551724  0.18750  0.444444  0.685841  0.732794   

           3par       ftr      orb%      drb%      trb%      ast%      stl%  \
0      0.557461  0.618980  0.426523  0.641577  0.533149  0.531587  0.242991   
1      0.322470  0.280453  0.517921  0.602151  0.591160  0.690293  0.387850   
2      0.288165  0.290368  0.503584  0.700717  0.486188  0.223421  0.439252   
3      0.283019  0.104816  0.250896  0.700717  0.444751  0.331279  0.369159   
4      0.185249  0.215297  0.489247  0.715054  0.657459  0.371341  0.401869   
...         ...       ...       ...       ...       ...       ...       ...   
11172  0.325901  0.294618  0.571685  0.575269  0.549724  0.482280  0.434579   
11173  0.540309  0.014164  0.213262  0.419355  0.135359  0.500770  0.453271   
11174  0.447684  0.593484  0.580645  0.786738  0.864641  0.394453  0.149533   
11175  0.480274  0.141643  0.437276  0.693548  0.500000  0.531587  0.523364   
11176  0.607204  0.236544  0.306452  0.562724  0.500000  0.611710  0.523364   

           blk%      tov%  usg%      ortg      drtg   fgmaxes  fgamaxes  \
0      0.000000  0.268657   0.0  0.660793  0.546256  0.363636  0.270270   
1      0.259901  0.391791   0.0  0.496696  0.417401  0.272727  0.351351   
2      0.089109  0.406716   0.0  0.348018  0.481278  0.136364  0.216216   
3      0.133663  0.708955   0.0 

## Construction of Future Games
We need to now construct a subset of the dataframe to reflect future games.

In [9]:
mapping = {'Brooklyn Nets': 'BRK', 'Golden State Warriors': 'GSW', 'Los Angeles Lakers': 'LAL',
       'Milwaukee Bucks': 'MIL', 'Boston Celtics': 'BOS', 'Charlotte Hornets': 'CHO',
       'Chicago Bulls': 'CHI', 'Cleveland Cavaliers': 'CLE', 'Denver Nuggets': 'DEN',
       'Detroit Pistons': 'DET', 'Houston Rockets': 'HOU', 'Indiana Pacers': 'IND',
       'Memphis Grizzlies': 'MEM', 'Minnesota Timberwolves': 'MIN',
       'New Orleans Pelicans': 'NOP', 'New York Knicks': 'NYK', 'Oklahoma City Thunder': 'OKC',
       'Orlando Magic': 'ORL', 'Philadelphia 76ers': 'PHI', 'Phoenix Suns': 'PHO',
       'Portland Trail Blazers': 'POR', 'Sacramento Kings': 'SAC', 'San Antonio Spurs': 'SAS',
       'Toronto Raptors': 'TOR', 'Utah Jazz': 'UTA', 'Washington Wizards': 'WAS',
       'Atlanta Hawks': 'ATL', 'Dallas Mavericks': 'DAL', 'LA Clippers': 'LAC', 'Miami Heat': 'MIA'}

In [10]:
# Need to update this 
future_teams = ['NYK', 'MIL', 'PHI', 'SAS', 'SAC', 'BRK', 'UTA', 'GSW', 'MIN', 'LAL']

In [11]:
def construction(df, teams):
    constructed = []
    # teams are the teams that will play in the future
    for team in teams:
        constructed.append(df[df['Teams'] == team].iloc[-1])
    tonight = pd.concat(constructed, axis=1).T
    tonight = tonight.reset_index(drop=True)
    return tonight

def opponent(tonight):
    figure = []
    for i in range(len(tonight['Teams'])):
        if (i % 2 == 0):
            figure.append(tonight['Teams'][i+1])
        else:
            figure.append(tonight['Teams'][i-1])
    return figure

In [12]:
tonight = construction(full, future_teams)

In [13]:
number_of_games = (int)(len(future_teams) / 2)
tonight['home_next'] = [0,1] * number_of_games 
tonight['team_next_opp'] = opponent(tonight)
date_to_repeat = '2024-04-07' # need to update this
tonight['date_next'] = [date_to_repeat] * len(tonight)

In [14]:
# merge stats from opposing teams
complete = tonight.merge(tonight,
               left_on=['Teams', 'date_next'],
               right_on = ['team_next_opp', 'date_next'])

# concat the spreads
spread_df = spread(complete)
complete = pd.concat([complete, spread_df], axis=1)

# concat the odds
complete = pd.merge(complete, odds, on=['Teams_x', 'date_next', 'Teams_y'], how='left')
complete = complete.dropna()

In [16]:
complete

mp_x      fg_x     fga_x     fg%_x      3p_x     3pa_x     3p%_x      ft_x  \
0  0.0  0.340909  0.333333  0.394737  0.296296  0.316667   0.35705  0.380952   
1  0.0  0.454545  0.516667  0.399522   0.37037  0.533333  0.280724  0.357143   
2  0.0  0.454545  0.266667  0.598086  0.518519  0.483333  0.441138  0.380952   
3  0.0  0.477273  0.333333  0.566986  0.555556       0.6  0.389392  0.190476   
4  0.0  0.386364       0.6   0.27512  0.555556  0.683333  0.341527  0.119048   
5  0.0  0.386364  0.166667  0.602871  0.333333  0.283333  0.437257  0.571429   
6  0.0  0.318182  0.466667  0.277512  0.259259  0.333333  0.298836       0.5   
7  0.0  0.454545       0.5  0.411483   0.37037  0.416667  0.354463  0.238095   
8  0.0  0.272727  0.366667  0.284689  0.296296  0.316667   0.35705  0.214286   
9  0.0  0.545455  0.283333  0.703349  0.518519      0.45  0.469599  0.190476   

      fta_x     ft%_x     orb_x     drb_x     trb_x     ast_x     stl_x blk_x  \
0  0.411765  0.660787  0.310345  0.295455  0.307692  0.333333  0.409091   0.1   
1  0.352941  0.742198  0.413793  0.386364  0.442308  0.309524  0.363636   0.3   
2  0.392157  0.705563  0.310345  0.363636  0.365385  0.452381  0.545455  0.45   
3  0.196078  0.686567  0.206897  0.272727  0.230769  0.619048  0.409091   0.4   
4  0.196078  0.373134  0.517241  0.522727  0.615385  0.309524  0.181818  0.15   
5  0.588235  0.712347   0.37931  0.477273       0.5  0.285714  0.136364  0.35   
6  0.509804  0.719132  0.482759  0.295455  0.403846  0.285714       0.5   0.2   
7  0.294118  0.548168   0.37931  0.386364  0.423077  0.452381  0.545455  0.25   
8  0.215686  0.709634  0.275862  0.454545  0.423077  0.357143  0.409091   0.3   
9  0.254902   0.49118  0.206897  0.409091  0.346154   0.52381       0.5  0.55   

      tov_x     pf_x     pts_x     ts%_x    efg%_x    3par_x     ftr_x  \
0  0.551724  0.28125  0.296296  0.393805  0.390688  0.411664  0.362606   
1  0.413793   0.5625  0.398148  0.402655  0.402834   0.57976  0.269122   
2   0.62069      0.5  0.444444  0.652655  0.645749  0.660377  0.365439   
3  0.517241  0.21875  0.398148  0.597345  0.621457  0.763293  0.175637   
4  0.206897  0.28125  0.296296  0.267699  0.342105  0.696398   0.13881   
5  0.586207  0.15625  0.416667  0.641593  0.597166  0.447684  0.593484   
6  0.482759   0.3125  0.314815  0.300885  0.269231  0.379074  0.405099   
7  0.482759  0.40625  0.351852  0.373894  0.412955  0.457976  0.228045   
8  0.586207   0.4375  0.175926  0.267699  0.295547  0.397942  0.186969   
9  0.551724   0.1875  0.444444  0.685841  0.732794  0.607204  0.236544   

     orb%_x    drb%_x    trb%_x    ast%_x    stl%_x    blk%_x    tov%_x  \
0  0.342294  0.290323  0.223757  0.525424  0.429907  0.071782  0.537313   
1  0.430108  0.616487  0.469613  0.372881  0.369159  0.346535  0.380597   
2  0.424731  0.428315  0.450276  0.599384  0.579439  0.366337  0.619403   
3  0.299283  0.494624  0.370166  0.832049  0.439252  0.346535  0.533582   
4  0.498208  0.578853  0.538674  0.436055  0.205607  0.165842  0.197761   
5  0.580645  0.786738  0.864641  0.394453  0.149533  0.326733  0.600746   
6   0.44086  0.612903  0.381215   0.46379       0.5  0.175743  0.432836   
7  0.448029  0.650538  0.549724  0.599384  0.556075  0.247525   0.44403   
8  0.286738  0.707885  0.425414  0.657935  0.425234   0.29703  0.578358   
9  0.306452  0.562724       0.5   0.61171  0.523364   0.49505  0.570896   

  usg%_x    ortg_x    drtg_x fgmaxes_x fgamaxes_x fg%maxes_x 3pmaxes_x  \
0    0.0  0.377753  0.468062  0.454545   0.486486      0.375  0.230769   
1    0.0  0.460352   0.52533  0.318182   0.378378      0.445  0.230769   
2    0.0  0.566079  0.338106  0.272727   0.081081   0.666667  0.153846   
3    0.0  0.518722  0.495595  0.181818   0.135135      0.445  0.230769   
4    0.0  0.446035   0.45815  0.590909   0.513514   0.253333  0.461538   
5    0.0  0.581498  0.463656  0.318182   0.405405   0.761667  0.153846   
6    0.0  0.337004  0.647577  0.136364   0.108108      0.445  0.076923

In [17]:
# load models and features
save_path = '/Users/benjamincheng/Documents/GitHub/Sports-Betting/ml_notebooks/factors/'
file_name = 'calibrated_predictors_ridge_classifier_30_predictors_19.97%_2019_2024.txt'
file_path = save_path + file_name
with open(file_path, 'r') as f:
    predictors = f.read()
    
# model
save_path = '/Users/benjamincheng/Documents/GitHub/Sports-Betting/ml_notebooks/weights/'
file_name = 'calibrated_ridge_classifier_30_predictors_19.97%_2019_2024.pkl'
file_path = save_path + file_name
with open(file_path, 'rb') as f:
    clf = pickle.load(f)

predictors = predictors.split(',')[:-1]
X = complete[predictors]
predictions = clf.predict(X)
result = pd.Series(predictions, index = X.index)
final = pd.concat([complete['Teams_x'], result], axis=1)

In [18]:
predictors

['usg%_x',
 '+/-maxes_x',
 'winrate_opp_x',
 'orbmaxes_3_x',
 'blkmaxes_3_x',
 'ts%_opp_3_x',
 'stl%maxes_opp_3_x',
 'trb%_5_x',
 'trb%_opp_5_x',
 'stl%maxes_opp_5_x',
 'drtgmaxes_opp_5_x',
 'astmaxes_opp_15_x',
 'ft%maxes_exp_5_x',
 'stl%maxes_opp_exp_9_x',
 'winrate_opp_exp_9_x',
 'fg_exp_12_x',
 'trb_exp_12_x',
 'ts%maxes_exp_12_x',
 'FGA_base_x',
 'FGA_RANK_base_x',
 'drb%maxes_y',
 '3pamaxes_3_y',
 'stl%maxes_opp_3_y',
 'fg%_5_y',
 'efg%_exp_5_y',
 'blk%maxes_exp_12_y',
 'FGA_RANK_base_y',
 'FGA_RANK_base_spread',
 'Draftkings_odds_x',
 'Fanduel_odds_y']

In [19]:
probabilities = clf.predict_proba(X)[:, 1]
result = pd.DataFrame({
    'Predictions': predictions,
    'Probabilities': probabilities,
    'Fanduel': complete['Fanduel_odds_x'],
    'Draftkings': complete['Draftkings_odds_x'],
    'BetMGM': complete['BetMGM_odds_x']
}, index=X.index)

# Concatenate the predictions and probabilities with the original data
final = pd.concat([complete['date_next'],complete['Teams_x'], result], axis=1)
final.columns = ['Date','Teams', 'Predictions', 'Probabilities', 'Fanduel', 'Draftkings', 'BetMGM']
# final.loc[final['Teams'] == 'POR', 'BetMGM'] = 1.90
# final.loc[final['Teams'] == 'POR', 'Draftkings'] = 1.90
# final.loc[final['Teams'] == 'POR', 'Fanduel'] = 1.90
final['Our_Odds'] = 1 / final['Probabilities']
final['Best_Odds'] = final[['Fanduel', 'Draftkings', 'BetMGM']].max(axis=1)

In [20]:
from sklearn.metrics import accuracy_score
# final['Actual'] = [0,1,1,0,0,1,0,1,0,1,0,1,1,0,0,1,]
# accuracy = accuracy_score(final['Actual'], final['Predictions'])
# accuracy

In [21]:
final

,Date,Teams,Predictions,Probabilities,Fanduel,Draftkings,BetMGM,Our_Odds,Best_Odds
0,2024-04-07,NYK,0,0.395455,2.52,2.50,2.55,2.528736,2.55
1,2024-04-07,MIL,1,0.620690,1.56,1.56,1.53,1.611111,1.56
2,2024-04-07,PHI,1,0.595238,1.56,1.54,1.54,1.680000,1.56
3,2024-04-07,SAS,0,0.395455,2.52,2.54,2.50,2.528736,2.54
4,2024-04-07,SAC,1,0.785714,1.18,1.19,1.18,1.272727,1.19
5,2024-04-07,BRK,0,0.180000,5.10,4.90,5.00,5.555556,5.10
6,2024-04-07,UTA,0,0.180000,6.40,6.25,6.50,5.555556,6.50
7,2024-04-07,GSW,1,0.785714,1.13,1.13,1.12,1.272727,1.13
8,2024-04-07,MIN,1,0.581818,1.96,1.91,1.95,1.718750,1.96
9,2024-04-07,LAL,1,0.565641,1.89,1.91,1.87,1.767905,1.91


In [22]:
win = final[final['Predictions'] == 1]
bet = win[win['Our_Odds'] < win['Best_Odds']]
bet['odds_diff'] = bet['Best_Odds'] - bet['Our_Odds']

/var/folders/99/zv3w4flx1598dxlknc099kfr0000gn/T/ipykernel_48072/3053637661.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bet['odds_diff'] = bet['Best_Odds'] - bet['Our_Odds']


In [23]:
bet

,Date,Teams,Predictions,Probabilities,Fanduel,Draftkings,BetMGM,Our_Odds,Best_Odds,odds_diff
8,2024-04-07,MIN,1,0.581818,1.96,1.91,1.95,1.718750,1.96,0.241250
9,2024-04-07,LAL,1,0.565641,1.89,1.91,1.87,1.767905,1.91,0.142095


In [24]:
def kelly(decimal_odds, P, fractional):
    # shrinkage is the percent we want to shrink bets to account for uncertanity
    B = decimal_odds - 1
    Q = 1 - P
    solution = (B*P - Q) / B
    solution = solution * fractional
    return solution

In [25]:
bet['Kelly'] = kelly(bet['Best_Odds'], bet['Probabilities'], 0.50)

/var/folders/99/zv3w4flx1598dxlknc099kfr0000gn/T/ipykernel_48072/2290681703.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bet['Kelly'] = kelly(bet['Best_Odds'], bet['Probabilities'], 0.50)


In [26]:
bet_very_undervalued = bet[bet['odds_diff'] > 0.06]
bet_very_undervalued = bet_very_undervalued[bet_very_undervalued['Best_Odds'] > 1.1]

In [27]:
bet_very_undervalued

,Date,Teams,Predictions,Probabilities,Fanduel,Draftkings,BetMGM,Our_Odds,Best_Odds,odds_diff,Kelly
8,2024-04-07,MIN,1,0.581818,1.96,1.91,1.95,1.718750,1.96,0.241250,0.073106
9,2024-04-07,LAL,1,0.565641,1.89,1.91,1.87,1.767905,1.91,0.142095,0.044162


In [29]:
.073106 * 6500

475.189